# WiDS 2026 Data Preparation Notebook

What this notebook does:
1. Loads raw files from `data/`
2. Explores the data with simple charts
3. Cleans missing values and handles extreme values
4. Builds new useful features
5. Removes redundant features
6. Creates train and validation splits
7. Scales features for models that need scaling
8. Saves all processed files for modeling


## 1. Setup and imports

We import common data science libraries and define a few constants used in the rest of the notebook.


In [ ]:
# If you are in Colab and packages are missing, uncomment and run this line.
# !pip install -q pandas numpy matplotlib seaborn scikit-learn

import os
import json
import pickle
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 200)
sns.set_theme(style='whitegrid')

RANDOM_STATE = 42
DATA_DIR = Path('data')


## 2. Load data

We load train, test, metadata, and sample submission files. Then we print shapes, data types, and sample rows.


In [ ]:
train_raw = pd.read_csv(DATA_DIR / 'train.csv')
test_raw = pd.read_csv(DATA_DIR / 'test.csv')
submission_template = pd.read_csv(DATA_DIR / 'sample_submission.csv')
metadata = pd.read_csv(DATA_DIR / 'metaData.csv')

print('Train shape:', train_raw.shape)
print('Test shape:', test_raw.shape)
print('Submission template shape:', submission_template.shape)
print('Metadata shape:', metadata.shape)

id_col = 'event_id'
target_cols = ['time_to_hit_hours', 'event']
base_feature_cols = [c for c in train_raw.columns if c not in [id_col] + target_cols]

print('')
print('Number of base features:', len(base_feature_cols))
print('Base feature columns:')
print(base_feature_cols)

for name, df in [('train', train_raw), ('test', test_raw), ('sample_submission', submission_template), ('metaData', metadata)]:
    print('')
    print(f"{name.upper()} - dtypes")
    print(df.dtypes)
    print('')
    print(f"{name.upper()} - first 5 rows")
    display(df.head())


### What to look for

The key thing to confirm here is that train has target columns (`time_to_hit_hours`, `event`) and test does not. The feature columns should match between train and test.


## 3. Exploratory data analysis

These charts help us understand class balance, missing data, feature shapes, and train versus test differences.


### 3.1 Target distribution

We compare how `time_to_hit_hours` looks for events that did happen versus censored events, and we check the event ratio.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ev, color, label in [(1, '#d95f02', 'Event happened (event=1)'), (0, '#1b9e77', 'Censored (event=0)')]:
    subset = train_raw.loc[train_raw['event'] == ev, 'time_to_hit_hours']
    axes[0].hist(subset, bins=20, alpha=0.65, color=color, label=label)

axes[0].set_title('Distribution of time_to_hit_hours by event status')
axes[0].set_xlabel('time_to_hit_hours')
axes[0].set_ylabel('Count')
axes[0].legend()

train_raw['event'].value_counts().sort_index().plot(kind='bar', ax=axes[1], color=['#1b9e77', '#d95f02'])
axes[1].set_title('Event indicator counts (censoring ratio)')
axes[1].set_xlabel('event (0=censored, 1=hit)')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

event_rate = train_raw['event'].mean()
print(f'Event rate (event=1): {event_rate:.3f}')
print(f'Censoring rate (event=0): {1 - event_rate:.3f}')


### Insight

If one class is much larger than the other, model evaluation can be misleading. That is why we will stratify the train and validation split by `event` later.


### 3.2 Missing values

We calculate missing value percentages in train and test and plot them side by side.


In [ ]:
missing_train_pct = train_raw.isna().mean().mul(100)
missing_test_pct = test_raw.isna().mean().mul(100)

missing_compare = pd.DataFrame({
    'train_missing_pct': missing_train_pct,
    'test_missing_pct': missing_test_pct.reindex(train_raw.columns, fill_value=np.nan)
}).fillna(0).sort_values('train_missing_pct', ascending=False)

display(missing_compare.head(15))

plot_df = missing_compare[(missing_compare['train_missing_pct'] > 0) | (missing_compare['test_missing_pct'] > 0)]

if len(plot_df) > 0:
    ax = plot_df.plot(kind='bar', figsize=(14, 5), color=['#4c72b0', '#dd8452'])
    ax.set_title('Missing value percentage by column')
    ax.set_xlabel('Column')
    ax.set_ylabel('Missing percentage')
    plt.xticks(rotation=75, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print('No missing values found in train or test.')


### Insight

Columns with small missing percentages can be filled safely. Columns with very high missing percentages should be reviewed carefully before modeling.


### 3.3 Feature distributions

We plot histograms for all base features to see scale differences and skewed variables.


In [ ]:
num_features = len(base_feature_cols)
n_cols = 4
n_rows = int(np.ceil(num_features / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4 * n_rows))
axes = np.array(axes).reshape(-1)

for i, col in enumerate(base_feature_cols):
    sns.histplot(train_raw[col], bins=20, kde=False, color='#4c72b0', ax=axes[i])
    axes[i].set_title(col)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')

for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


### Insight

Many wildfire variables are usually skewed, which is normal for growth and distance data. Later we add transformed features to help models read these patterns better.


### 3.4 Correlation analysis

We inspect feature correlations with target columns and flag very similar feature pairs.


In [ ]:
corr_matrix = train_raw[base_feature_cols + target_cols].corr(numeric_only=True)

plt.figure(figsize=(10, 12))
sns.heatmap(corr_matrix[["time_to_hit_hours", "event"]].sort_values('event', ascending=False),
            cmap='coolwarm', center=0)
plt.title('Feature correlation with time_to_hit_hours and event')
plt.xlabel('Targets')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

feature_corr_abs = train_raw[base_feature_cols].corr(numeric_only=True).abs()
upper = feature_corr_abs.where(np.triu(np.ones(feature_corr_abs.shape), k=1).astype(bool))
high_corr_pairs = [
    (c1, c2, upper.loc[c1, c2])
    for c1 in upper.index
    for c2 in upper.columns
    if pd.notna(upper.loc[c1, c2]) and upper.loc[c1, c2] > 0.90
]

high_corr_df = pd.DataFrame(high_corr_pairs, columns=['feature_1', 'feature_2', 'abs_corr'])    .sort_values('abs_corr', ascending=False)

print(f'Number of feature pairs with |r| > 0.90: {len(high_corr_df)}')
display(high_corr_df.head(20))


### Insight

If two features carry almost the same information, keeping both can make some models unstable. We will remove very redundant pairs later.


### 3.5 Train versus test distribution check

We compare train and test for 10 features with the largest standardized mean difference.


In [ ]:
mean_shift = ((train_raw[base_feature_cols].mean() - test_raw[base_feature_cols].mean()).abs() /
              (train_raw[base_feature_cols].std().replace(0, np.nan))).fillna(0)
top_shift_features = mean_shift.sort_values(ascending=False).head(10).index.tolist()

print('Top 10 potential shift features:')
print(top_shift_features)

fig, axes = plt.subplots(2, 5, figsize=(22, 8))
axes = axes.flatten()

for idx, col in enumerate(top_shift_features):
    sns.kdeplot(train_raw[col], ax=axes[idx], label='Train', color='#4c72b0', fill=True, alpha=0.2)
    sns.kdeplot(test_raw[col], ax=axes[idx], label='Test', color='#dd8452', fill=True, alpha=0.2)
    axes[idx].set_title(col)
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Density')
    if idx == 0:
        axes[idx].legend()

plt.tight_layout()
plt.show()


### Insight

If train and test curves are very different for a feature, that feature may behave differently at prediction time. This is a useful warning for model tuning and validation strategy.


## 4. Data cleaning

We define reusable cleaning functions and create preview cleaned datasets. Final leak-safe fitting of cleaning parameters is done in the split section.


### 4.1 Build cleaning helpers and review missing counts

Rules:
1. Missing below 5 percent: fill with train median
2. Missing above 20 percent: flag for review
3. Outliers: clip to 1st and 99th percentile, do not remove rows
4. Drop zero-variance features


In [ ]:
def fit_cleaning_params(train_df, feature_cols):
    params = {}

    medians = train_df[feature_cols].median(numeric_only=True)
    missing_pct = train_df[feature_cols].isna().mean()
    high_missing_cols = missing_pct[missing_pct > 0.20].index.tolist()

    numeric_cols = train_df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()

    clip_bounds = {}
    for col in numeric_cols:
        lower = train_df[col].quantile(0.01)
        upper = train_df[col].quantile(0.99)
        clip_bounds[col] = (lower, upper)

    zero_var_cols = [col for col in feature_cols if train_df[col].nunique(dropna=False) <= 1]

    outlier_flags = {}
    for col in numeric_cols:
        q1 = train_df[col].quantile(0.25)
        q3 = train_df[col].quantile(0.75)
        iqr = q3 - q1
        if iqr == 0:
            outlier_flags[col] = 0
            continue
        lower = q1 - 3 * iqr
        upper = q3 + 3 * iqr
        outlier_flags[col] = int(((train_df[col] < lower) | (train_df[col] > upper)).sum())

    params['medians'] = medians.to_dict()
    params['high_missing_cols'] = high_missing_cols
    params['numeric_cols'] = numeric_cols
    params['clip_bounds'] = clip_bounds
    params['zero_var_cols'] = zero_var_cols
    params['outlier_flags'] = outlier_flags
    return params


def apply_cleaning(df, feature_cols, params, include_event=False):
    cleaned = df.copy()

    # Fill missing values with training medians
    for col in feature_cols:
        if col in params['medians']:
            cleaned[col] = cleaned[col].fillna(params['medians'][col])

    # Winsorization using train-based percentiles
    for col in params['numeric_cols']:
        if col in cleaned.columns:
            lo, hi = params['clip_bounds'][col]
            cleaned[col] = cleaned[col].clip(lo, hi)

    # Remove zero variance features
    drop_cols = [c for c in params['zero_var_cols'] if c in cleaned.columns]
    if drop_cols:
        cleaned = cleaned.drop(columns=drop_cols)

    # Enforce integer dtypes for indicator columns
    if 'low_temporal_resolution_0_5h' in cleaned.columns:
        cleaned['low_temporal_resolution_0_5h'] = cleaned['low_temporal_resolution_0_5h'].round().astype(int)

    if include_event and 'event' in cleaned.columns:
        cleaned['event'] = cleaned['event'].round().astype(int)

    return cleaned


missing_counts = pd.DataFrame({
    'train_missing_count': train_raw.isna().sum(),
    'test_missing_count': test_raw.isna().sum().reindex(train_raw.columns, fill_value=np.nan)
}).fillna(0).astype(int)

display(missing_counts[missing_counts.sum(axis=1) > 0].sort_values('train_missing_count', ascending=False).head(20))

cleaning_preview_params = fit_cleaning_params(train_raw, base_feature_cols)

print('Columns with >20% missing in train:', cleaning_preview_params['high_missing_cols'])
print('Zero variance columns:', cleaning_preview_params['zero_var_cols'])

outlier_flag_df = pd.DataFrame({
    'feature': list(cleaning_preview_params['outlier_flags'].keys()),
    'outlier_count_3xIQR': list(cleaning_preview_params['outlier_flags'].values())
}).sort_values('outlier_count_3xIQR', ascending=False)

display(outlier_flag_df.head(15))

train_clean_preview = apply_cleaning(train_raw, base_feature_cols, cleaning_preview_params, include_event=True)
test_clean_preview = apply_cleaning(test_raw, base_feature_cols, cleaning_preview_params, include_event=False)

clean_base_feature_cols = [c for c in base_feature_cols if c not in cleaning_preview_params['zero_var_cols']]
print('Base feature count after zero-variance removal:', len(clean_base_feature_cols))


### Insight

The cleaning rules are simple and repeatable. This makes the pipeline easier to audit and less likely to break when rerun.


## 5. Feature engineering

We create extra features that combine growth, distance, direction, and time signals.


### 5.1 Create engineered features for train and test

Each engineered feature is created with the same formula in every dataset.


In [ ]:
def safe_divide(a, b):
    return a / (b.replace(0, np.nan) if isinstance(b, pd.Series) else (b if b != 0 else np.nan))


def engineer_features(df):
    out = df.copy()

    # Cyclical encodings for temporal columns
    cyclical_specs = [
        ('event_start_hour', 24),
        ('event_start_dayofweek', 7),
        ('event_start_month', 12),
    ]

    for col, period in cyclical_specs:
        if col in out.columns:
            out[f'{col}_sin'] = np.sin(2 * np.pi * out[col] / period)
            out[f'{col}_cos'] = np.cos(2 * np.pi * out[col] / period)
            out = out.drop(columns=[col])

    # Interaction features
    out['threat_score'] = out['closing_speed_m_per_h'] * out['alignment_abs']
    out['growth_threat'] = out['area_growth_rate_ha_per_h'] * out['closing_speed_m_per_h']
    out['directional_advance'] = out['along_track_speed'] * out['alignment_cos']
    out['area_distance_ratio'] = safe_divide(out['area_first_ha'], out['dist_min_ci_0_5h'] + 1).fillna(0)
    out['speed_per_area'] = safe_divide(out['centroid_speed_m_per_h'], out['area_first_ha'] + 1).fillna(0)
    out['radial_vs_centroid_speed'] = safe_divide(out['radial_growth_rate_m_per_h'], out['centroid_speed_m_per_h'] + 1).fillna(0)

    # Nonlinear transforms
    out['log_dist_min'] = np.log1p(np.clip(out['dist_min_ci_0_5h'], a_min=0, a_max=None))
    out['sqrt_area_first'] = np.sqrt(np.clip(out['area_first_ha'], a_min=0, a_max=None))
    out['closing_speed_sq'] = out['closing_speed_m_per_h'] ** 2

    # Temporal resolution interactions
    out['growth_quality'] = out['area_growth_rate_ha_per_h'] * (1 - out['low_temporal_resolution_0_5h'])
    out['speed_quality'] = out['closing_speed_m_per_h'] * out['dist_fit_r2_0_5h']

    # Survival-focused heuristics
    out['time_to_close_estimate'] = safe_divide(out['dist_min_ci_0_5h'], out['closing_speed_abs_m_per_h'] + 1).fillna(200)
    out['time_to_close_estimate'] = out['time_to_close_estimate'].clip(0, 200)

    out['is_closing'] = (out['closing_speed_m_per_h'] > 0).astype(int)
    out['is_aligned'] = (out['alignment_abs'] > 0.5).astype(int)
    out['immediate_threat'] = ((out['is_closing'] == 1) & (out['is_aligned'] == 1)).astype(int)

    return out


train_fe_preview = engineer_features(train_clean_preview)
test_fe_preview = engineer_features(test_clean_preview)

id_and_targets = [id_col] + target_cols
engineered_feature_cols = [
    c for c in train_fe_preview.columns
    if c not in train_clean_preview.columns and c not in id_and_targets
]

print('Engineered feature count:', len(engineered_feature_cols))
print(engineered_feature_cols)


### 5.2 Engineered feature summary table

This quick table gives a plain-language reminder of what each new feature means.


In [ ]:
engineered_descriptions = {
    'event_start_hour_sin': 'Circular encoding of start hour (sine)',
    'event_start_hour_cos': 'Circular encoding of start hour (cosine)',
    'event_start_dayofweek_sin': 'Circular encoding of day of week (sine)',
    'event_start_dayofweek_cos': 'Circular encoding of day of week (cosine)',
    'event_start_month_sin': 'Circular encoding of month (sine)',
    'event_start_month_cos': 'Circular encoding of month (cosine)',
    'threat_score': 'Closing speed times direction alignment',
    'growth_threat': 'Growth rate times closing speed',
    'directional_advance': 'Along-track speed weighted by direction sign',
    'area_distance_ratio': 'Initial area divided by distance to evac zone',
    'speed_per_area': 'Centroid speed relative to area',
    'radial_vs_centroid_speed': 'Spread speed relative to centroid movement speed',
    'log_dist_min': 'Log-scaled minimum distance to evac zone',
    'sqrt_area_first': 'Square-root transform of initial area',
    'closing_speed_sq': 'Squared closing speed to stress larger values',
    'growth_quality': 'Growth rate discounted when temporal resolution is low',
    'speed_quality': 'Closing speed weighted by linear-fit quality',
    'time_to_close_estimate': 'Simple time estimate to reach evac zone',
    'is_closing': 'Binary flag for positive closing speed',
    'is_aligned': 'Binary flag for alignment above 0.5',
    'immediate_threat': 'Binary flag when both closing and aligned',
}

feature_summary = pd.DataFrame([
    {'feature': f, 'description': engineered_descriptions.get(f, 'Engineered feature')}
    for f in engineered_feature_cols
]).sort_values('feature')

display(feature_summary)


### Insight

These engineered features package raw measurements into easier risk signals, such as "moving closer quickly" or "large fire already near an evacuation zone".


## 6. Feature selection

Now we remove highly redundant features and keep a cleaner feature set.


### 6.1 Correlation-based redundancy removal

If two features have absolute correlation above 0.95, we keep the one that is more correlated with `event` in the training data.


In [ ]:
def select_features_by_correlation(train_df, candidate_cols, target_col='event', threshold=0.95):
    corr_abs = train_df[candidate_cols].corr(numeric_only=True).abs()
    upper = corr_abs.where(np.triu(np.ones(corr_abs.shape), k=1).astype(bool))

    target_corr = train_df[candidate_cols].corrwith(train_df[target_col]).abs().fillna(0)

    to_drop = set()
    decisions = []

    for col in upper.columns:
        high_corr_with_col = upper.index[upper[col] > threshold].tolist()
        for row_col in high_corr_with_col:
            if row_col in to_drop or col in to_drop:
                continue

            row_score = target_corr.get(row_col, 0)
            col_score = target_corr.get(col, 0)

            if row_score < col_score:
                drop_col, keep_col = row_col, col
            elif col_score < row_score:
                drop_col, keep_col = col, row_col
            else:
                drop_col, keep_col = sorted([row_col, col])[1], sorted([row_col, col])[0]

            to_drop.add(drop_col)
            decisions.append({
                'feature_kept': keep_col,
                'feature_dropped': drop_col,
                'pair_abs_corr': upper.loc[row_col, col],
                'kept_abs_corr_with_event': target_corr.get(keep_col, 0),
                'dropped_abs_corr_with_event': target_corr.get(drop_col, 0),
            })

    selected = [c for c in candidate_cols if c not in to_drop]
    decisions_df = pd.DataFrame(decisions).sort_values('pair_abs_corr', ascending=False) if decisions else pd.DataFrame()
    return selected, decisions_df


preview_candidate_cols = [c for c in train_fe_preview.columns if c not in [id_col] + target_cols]
preview_feature_cols, preview_drop_log = select_features_by_correlation(
    train_fe_preview,
    preview_candidate_cols,
    target_col='event',
    threshold=0.95,
)

print('Candidate feature count before pruning:', len(preview_candidate_cols))
print('Feature count after pruning:', len(preview_feature_cols))

if len(preview_drop_log) > 0:
    display(preview_drop_log.head(20))
else:
    print('No highly redundant pairs were removed at the 0.95 threshold.')


### Insight

This step keeps important information while reducing overlap. Fewer redundant columns can make models easier to train and explain.


## 7. Train and validation split

This section builds final modeling datasets with leakage safeguards:
1. Split raw train into train and validation with stratification
2. Fit cleaning and feature selection only on train split
3. Apply the same learned rules to validation and test
4. Create 5-fold stratified CV indices


### 7.1 Build leak-safe processed train, validation, and test datasets


In [ ]:
# Primary split on raw labeled data
raw_train_split, raw_val_split = train_test_split(
    train_raw,
    test_size=0.2,
    stratify=train_raw['event'],
    random_state=RANDOM_STATE,
)

# Fit cleaning rules on training split only
clean_params = fit_cleaning_params(raw_train_split, base_feature_cols)

train_clean = apply_cleaning(raw_train_split, base_feature_cols, clean_params, include_event=True)
val_clean = apply_cleaning(raw_val_split, base_feature_cols, clean_params, include_event=True)
test_clean = apply_cleaning(test_raw, base_feature_cols, clean_params, include_event=False)

# Apply engineering to all sets
train_fe = engineer_features(train_clean)
val_fe = engineer_features(val_clean)
test_fe = engineer_features(test_clean)

# Fit feature pruning on training split only
train_candidate_cols = [c for c in train_fe.columns if c not in [id_col] + target_cols]
feature_cols, drop_log = select_features_by_correlation(
    train_fe,
    train_candidate_cols,
    target_col='event',
    threshold=0.95,
)

# Final processed datasets
train_df = train_fe[[id_col] + feature_cols + target_cols].copy()
val_df = val_fe[[id_col] + feature_cols + target_cols].copy()
test_df = test_fe[[id_col] + feature_cols].copy()

# Combined processed train for CV folds (all labeled rows)
processed_train = pd.concat([train_df, val_df], axis=0, ignore_index=True)

# 5-fold stratified CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
folds = list(skf.split(processed_train[feature_cols], processed_train['event']))

print('Final feature count:', len(feature_cols))
print('Train shape:', train_df.shape)
print('Validation shape:', val_df.shape)
print('Test shape:', test_df.shape)

print('')
print('Dropped redundant features (first 15 rows):')
if len(drop_log) > 0:
    display(drop_log.head(15))
else:
    print('No features dropped in this stage.')


### 7.2 Validation checks

We verify stratification quality, overlap safety, shape consistency, and missing values.


In [ ]:
train_event_rate = train_df['event'].mean()
val_event_rate = val_df['event'].mean()
event_rate_gap = abs(train_event_rate - val_event_rate)

train_ids = set(train_df[id_col])
val_ids = set(val_df[id_col])
overlap_ids = train_ids.intersection(val_ids)

assert event_rate_gap <= 0.05, f'Event rate gap too large: {event_rate_gap:.4f}'
assert len(overlap_ids) == 0, 'Found overlapping event_id values between train and validation.'
assert list(train_df[feature_cols].columns) == list(val_df[feature_cols].columns) == list(test_df[feature_cols].columns), 'Feature columns are not aligned.'
assert train_df[feature_cols].isna().sum().sum() == 0, 'NaN found in train features.'
assert val_df[feature_cols].isna().sum().sum() == 0, 'NaN found in validation features.'
assert test_df[feature_cols].isna().sum().sum() == 0, 'NaN found in test features.'

print('Event rate in train:', round(train_event_rate, 4))
print('Event rate in validation:', round(val_event_rate, 4))
print('Absolute event rate gap:', round(event_rate_gap, 4))
print('Overlapping event_id count:', len(overlap_ids))
print('Feature alignment check: PASSED')
print('NaN checks: PASSED')
print('Number of CV folds:', len(folds))


### Insight

This is the most important quality gate in the notebook. If these checks pass, the data is consistent for modeling and less likely to leak information across splits.


## 8. Feature scaling

Some models need standardized features. We fit `StandardScaler` on train only and apply it to validation and test.


### 8.1 Fit and apply scaler


In [ ]:
scaler = StandardScaler()

train_scaled = scaler.fit_transform(train_df[feature_cols])
val_scaled = scaler.transform(val_df[feature_cols])
test_scaled = scaler.transform(test_df[feature_cols])

train_scaled_df = pd.DataFrame(train_scaled, columns=feature_cols, index=train_df.index)
val_scaled_df = pd.DataFrame(val_scaled, columns=feature_cols, index=val_df.index)
test_scaled_df = pd.DataFrame(test_scaled, columns=feature_cols, index=test_df.index)

print('Scaled train shape:', train_scaled_df.shape)
print('Scaled validation shape:', val_scaled_df.shape)
print('Scaled test shape:', test_scaled_df.shape)


## 9. Save processed outputs

We save unscaled files, scaled files, feature names, CV folds, and the scaler object for reuse.


### 9.1 Write all output artifacts to `data/`


In [ ]:
# Unscaled files
train_df.to_csv(DATA_DIR / 'train_processed.csv', index=False)
val_df.to_csv(DATA_DIR / 'val_processed.csv', index=False)
test_df.to_csv(DATA_DIR / 'test_processed.csv', index=False)

# Scaled files
train_scaled_df.to_csv(DATA_DIR / 'train_scaled.csv', index=False)
val_scaled_df.to_csv(DATA_DIR / 'val_scaled.csv', index=False)
test_scaled_df.to_csv(DATA_DIR / 'test_scaled.csv', index=False)

# Feature list
with open(DATA_DIR / 'feature_names.json', 'w') as f:
    json.dump(feature_cols, f, indent=2)

# CV folds
with open(DATA_DIR / 'cv_folds.pkl', 'wb') as f:
    pickle.dump(folds, f)

# Scaler object
with open(DATA_DIR / 'standard_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print('Saved files:')
print('- data/train_processed.csv')
print('- data/val_processed.csv')
print('- data/test_processed.csv')
print('- data/train_scaled.csv')
print('- data/val_scaled.csv')
print('- data/test_scaled.csv')
print('- data/feature_names.json')
print('- data/cv_folds.pkl')
print('- data/standard_scaler.pkl')


## 10. Final summary and sanity checks

We print final dataset stats and quick feature importance proxies using absolute correlation.


### 10.1 Final report


In [ ]:
original_feature_count = len(base_feature_cols)
final_feature_count = len(feature_cols)
engineered_in_final = [c for c in feature_cols if c not in base_feature_cols]

summary_rows = [
    ('Original base feature count', original_feature_count),
    ('Final selected feature count', final_feature_count),
    ('Engineered features kept', len(engineered_in_final)),
    ('Train rows', len(train_df)),
    ('Validation rows', len(val_df)),
    ('Test rows', len(test_df)),
    ('Train event rate', round(train_df['event'].mean(), 4)),
    ('Validation event rate', round(val_df['event'].mean(), 4)),
]

summary_df = pd.DataFrame(summary_rows, columns=['Metric', 'Value'])
display(summary_df)

corr_event = train_df[feature_cols + ['event']].corr(numeric_only=True)['event'].drop('event').abs().sort_values(ascending=False)
corr_time = train_df[feature_cols + ['time_to_hit_hours']].corr(numeric_only=True)['time_to_hit_hours'].drop('time_to_hit_hours').abs().sort_values(ascending=False)

print('Top 10 features by absolute correlation with event:')
display(corr_event.head(10).to_frame('abs_corr_with_event'))

print('Top 10 features by absolute correlation with time_to_hit_hours:')
display(corr_time.head(10).to_frame('abs_corr_with_time_to_hit_hours'))

no_nan_train = train_df[feature_cols].isna().sum().sum() == 0
no_nan_val = val_df[feature_cols].isna().sum().sum() == 0
no_nan_test = test_df[feature_cols].isna().sum().sum() == 0

print('No NaN in train features:', no_nan_train)
print('No NaN in validation features:', no_nan_val)
print('No NaN in test features:', no_nan_test)
print('Train and validation IDs disjoint:', set(train_df[id_col]).isdisjoint(set(val_df[id_col])))
print('Matching feature columns across splits:', list(train_df[feature_cols].columns) == list(val_df[feature_cols].columns) == list(test_df[feature_cols].columns))


### Done

The processed files are now ready for modeling. You can use unscaled files for tree-based methods and scaled files for linear or neural models.
